# Project 1 Part 4 - Creating a master parcel database

In this part of the project, we will use Python to read, process, and double all of the parcel data into a database.  Note that this is not our only alternative, and in Project 1 Part 4 b, we will look at another alternative, that is reading all the of original, raw files into their own database table, then using SQL to join/link/aggregate the tables.

## Chunking Files in Pandas – Part 1 (20 Points)

In this part of the project, you will use `Panda`’s to process the data from the MinneMUDAC 2016 competition Dive into Water Data.  The data can be found at the [MinneMUDAC site](http://minneanalytics.org/minnemudac/data/).  You should document your work in a Jupyter notebook, which will be used to submit your solution.  **For the rest of the parts of this project, we will limit ourselves to the years 2004-2014.**

1. Remind me why we want to skip 2003.

2003 has a lot less columns in common with the other years

2. Import the common columns list and translation dictionaries from the `.py` file you created in the last part of the project.

In [1]:
from project_data_Buske import common_columns

In [2]:
from project_data_Buske import latlong_to_code

In [3]:
from project_data_Buske import latlong_to_name

In [4]:
from project_data_Buske import code_to_name

In [5]:
from project_data_Buske import code_to_distance

In [6]:
from project_data_Buske import latlong_to_distance

3. Use glob and a list comprehension to get a list of file names for the years 2004-2014.

In [7]:
from glob import glob
files = glob('./data/MinneMUDAC_raw_files/20**_metro_tax_parcels.txt')[2:-1]
[file for file in files]

['./data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2005_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2006_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2007_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2008_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2009_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2010_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2011_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2012_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2013_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2014_metro_tax_parcels.txt']

In [ ]:
float_cols = ['EMV_BLDG', 'EMV_LAND', 'EMV_TOTAL']
string_cols = ['CITY', 'CITY_USPS', 'COOLING', 'DWELL_TYPE' ]
int_cols = ['COUNTY_ID' ]
nullable_int_cols = [ ]
date_cols = [ ]
messy_cols = ['BLOCK']

In [172]:
#parc = pd.read_csv('./data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt',sep = '|', dtype={'centroid_lat':str, 'centroid_long':str})

In [181]:
parc.columns

In [176]:
#list((sorted(common_columns)))

4. Use the first chunk of the first file to prototype an expression that <br>
    a. Selects the common columns <br>
    b. Fixes any issues with the column names <br>
    c. Changes columns to the correct types (if necessary).  More information about the columns can be found [here](ftp://ftp.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_prcls_open/metadata/metadata.html). It is **imperative** that you keep the lat and long columns as strings. <br>
    d. Use the translation dictionaries from the last part to add three new columns to the chunk: lake code, lake name, parcel distance to the lake.
    e. Filters to only properties that are within 1600 m (~1 mile) of the closest lake.

In [8]:
import pandas as pd
fileiterator = [pd.read_csv(file, chunksize = 50000, sep = '|', dtype={'centroid_lat':str, 'centroid_long':str}) for file in files]
fileiterator

In [9]:
from toolz import first

In [10]:
firstchunk = first(fileiterator[0])
firstchunk.head()

/Users/tr8934qw/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (38,67) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,...,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_lat,centroid_long
0,0.0,8.03,NaN,NaN,N,NaN,NaN,NaN,SAINT FRANCIS,NaN,...,NaN,NaN,NaN,NaN,1980.0,2004,NaN,NaN,45.41332,-93.26739
1,0.0,0.93,NaN,NaN,N,NaN,24457.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1974.0,2004,55005,NaN,45.41354,-93.2701
2,0.0,8.75,NaN,NaN,N,NaN,24442.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1969.0,2004,55005,NaN,45.41318,-93.27344
3,0.0,11.17,NaN,NaN,N,NaN,410.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1989.0,2004,55005,NaN,45.41167,-93.27684
4,0.0,14.46,NaN,NaN,N,NaN,480.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1995.0,2004,55070,NaN,45.41169,-93.27849


In [11]:
from dfply import *
from more_dfply import *

In [12]:
exp = (firstchunk 
         >> select(common_columns)
         >> mutate(lat_long = pd.Series(zip(firstchunk.centroid_lat, firstchunk.centroid_long)))
         >> mutate(lake_code = recode(X.lat_long, latlong_to_code))
         >> mutate(lake_name = recode(X.lat_long, latlong_to_name))
         >> mutate(parcel_dist_to_lake = recode(X.lat_long, latlong_to_distance))
         >> filter_by(~X.lake_name.isna())
         >> filter_by(X.parcel_dist_to_lake <= 1600)
      )         
exp

,OWN_ADD_L2,YEAR_BUILT,COOLING,GREEN_ACRE,PARC_CODE,EMV_TOTAL,SALE_DATE,TAX_ADD_L2,CITY_USPS,USE4_DESC,...,STREETTYPE,OWNER_NAME,centroid_lat,PREFIX_DIR,STREETNAME,OWN_ADD_L3,lat_long,lake_code,lake_name,parcel_dist_to_lake
42707,OAKDALE,0.0,NaN,N,0.0,209900.0,NaN,OAKDALE,NaN,NaN,...,NaN,NaN,45.21007,NaN,NaN,"MN, 55128","(45.21007, -93.0241)",02000400-01,Peltier Lake,777.247256
42708,HUGO,0.0,NaN,N,0.0,141520.0,2001-06-05,HUGO,NaN,NaN,...,NaN,NaN,45.20913,NaN,NaN,"MN, 55038","(45.20913, -93.03251)",02000400-01,Peltier Lake,777.247256
42709,HUGO,0.0,NaN,N,0.0,269290.0,1997-04-01,HUGO,NaN,NaN,...,NaN,NaN,45.20933,NaN,NaN,"MN, 55038","(45.20933, -93.03805)",02000400-01,Peltier Lake,777.247256
42710,STACY,0.0,NaN,N,0.0,14700.0,1991-06-08,STACY,NaN,NaN,...,NaN,NaN,45.20952,NaN,NaN,"MN, 55079","(45.20952, -93.0438)",02000400-01,Peltier Lake,777.247256
42711,ANOKA,0.0,NaN,N,0.0,148960.0,2003-10-02,ANOKA,NaN,NaN,...,NaN,NaN,45.20919,NaN,NaN,"MN, 55303","(45.20919, -93.04851)",02000400-01,Peltier Lake,777.247256
42712,ANOKA,0.0,NaN,N,0.0,147940.0,2003-10-02,ANOKA,NaN,NaN,...,NaN,NaN,45.20927,NaN,NaN,"MN, 55303","(45.20927, -93.05237)",02000400-01,Peltier Lake,777.247256
42713,ANOKA,0.0,NaN,N,0.0,147940.0,2003-10-02,ANOKA,NaN,NaN,...,NaN,NaN,45.2091,NaN,NaN,"MN, 55303","(45.2091, -93.05506)",02000400-01,Peltier Lake,777.247256
42714,HUGO,0.0,NaN,N,0.0,154700.0,NaN,HUGO,NaN,NaN,...,NaN,NaN,45.21015,NaN,NaN,"MN, 55038","(45.21015, -93.05759)",02000400-01,Peltier Lake,777.247256
42715,ST PAUL,0.0,NaN,N,0.0,7200.0,NaN,ST PAUL,NaN,NaN,...,NaN,NaN,45.20923,NaN,NaN,"MN, 55101","(45.20923, -93.06032)",02000400-01,Peltier Lake,777.247256
42716,ST PAUL,0.0,NaN,N,0.0,771215.0,NaN,ST PAUL,NaN,NaN,...,NaN,NaN,45.20659,NaN,NaN,"MN, 55101","(45.20659, -93.06247)",02000400-01,Peltier Lake,777.247256


In [2]:
cols_to_keep = ['ACRES_DEED',
                'ACRES_POLY',
                'AGPRE_ENRD',
                'AG_PRESERV',
                'BASEMENT',
                'CITY',
                'COOLING',
                'DWELL_TYPE',
                'EMV_BLDG',
                'EMV_LAND',
                'FIN_SQ_FT',
                'GARAGE', 
                'GARAGESQFT', 
                'GREEN_ACRE',
                'HOMESTEAD',
                'ID',
                'LANDMARK',
                'OWN_ADD_L1',
                'OWN_ADD_L2',
                'OWN_ADD_L3',
                'PARC_CODE',
                'PIN', 
                'SALE_VALUE', 
                'SPEC_ASSES',
                'TAX_CAPAC', 
                'TAX_EXEMPT', 
                'TOTAL_TAX', 
                'USE1_DESC',
                'USE2_DESC',
                'USE3_DESC',
                'USE4_DESC',
                'WSHD_DIST',
                'XUSE1_DESC',
                'XUSE2_DESC',
                'XUSE3_DESC',
                'XUSE4_DESC',
                'YEAR_BUILT',
                'Year',
                'centroid_lat',
                'centroid_long']
                

In [3]:
cols_to_drop = ['AGPRE_EXPD',#drop
                'BLDG_NUM',#drop
                'BLOCK',#drop
                'CITY_USPS',#drop
                'COUNTY_ID',#drop
                'EMV_TOTAL',#drop
                'HEATING', #drop
                'HOME_STYLE',#drop
                'LOT',#drop
                'MULTI_USES',#drop
                'NUM_UNITS',#drop
                'OPEN_SPACE',#drop
                'OWNER_MORE',#drop
                'OWNER_NAME',#drop
                'PLAT_NAME', #drop
                'PREFIXTYPE', #drop
                'PREFIX_DIR', #drop
                'SALE_DATE', #drop
                'SCHOOL_DST', #drop
                'STREETNAME', #drop
                'STREETTYPE', #drop
                'SUFFIX_DIR', #drop
                'Shape_Area', #drop
                'Shape_Leng', #drop
                'TAX_ADD_L1', #drop
                'TAX_ADD_L2', #drop
                'TAX_ADD_L3', #drop
                'TAX_NAME', #drop
                'UNIT_INFO', #drop
                'ZIP',#drop
                'ZIP4',#drop
                ]

In [22]:
from sqlalchemy import String, Float, Integer, DateTime
common_parcel_types = {'ACRES_DEED':Float,
                       'ACRES_POLY':Float,
                       'AGPRE_ENRD':DateTime,
                       'AGPRE_EXPD':DateTime,
                       'AG_PRESERV':String,
                       'BASEMENT':String,
                       'BLDG_NUM':Float,
                       'BLOCK':String,
                       'CITY':String,
                       'CITY_USPS':String,
                       'COOLING':String,
                       'COUNTY_ID':String,
                       'DWELL_TYPE':String,
                       'EMV_BLDG':String,
                       'EMV_LAND':String,
                       'EMV_TOTAL':String,
                       'FIN_SQ_FT':String,
                       'GARAGE':String,
                       'GARAGESQFT':String,
                       'GREEN_ACRE':String,
                       'HEATING':String,
                       'HOMESTEAD':String,
                       'HOME_STYLE':String,
                       'ID':String,
                       'LANDMARK':String,
                       'LOT':String,
                       'MULTI_USES':String,
                       'NUM_UNITS':String,
                       'OPEN_SPACE':String,
                       'OWNER_MORE':String,
                       'OWNER_NAME':String,
                       'OWN_ADD_L1':String,
                       'OWN_ADD_L2':String,
                       'OWN_ADD_L3':String,
                       'PARC_CODE':String,
                       'PIN':String,
                       'PLAT_NAME':String,
                       'PREFIXTYPE':String, 
                       'PREFIX_DIR':String,
                       'SALE_DATE':String, 
                       'SALE_VALUE':String,
                       'SCHOOL_DST':String, #drop
                       'SPEC_ASSES':String,
                       'STREETNAME':String, #drop
                       'STREETTYPE':String, #drop
                       'SUFFIX_DIR':String, #drop
                       'Shape_Area':String, #drop
                       'Shape_Leng':String, #drop
                       'TAX_ADD_L1':String, #drop
                       'TAX_ADD_L2':String, #drop
                       'TAX_ADD_L3':String, #drop
                       'TAX_CAPAC':String, #drop
                       'TAX_EXEMPT':String, #drop
                       'TAX_NAME':String, #drop
                       'TOTAL_TAX':String, #drop
                       'UNIT_INFO':String, #drop
                       'USE1_DESC':String,
                       'USE2_DESC':String,
                       'USE3_DESC':String,
                       'USE4_DESC':String,
                       'WSHD_DIST':String,
                       'XUSE1_DESC':String,
                       'XUSE2_DESC':String,
                       'XUSE3_DESC':String,
                       'XUSE4_DESC':String,
                       'YEAR_BUILT':String,
                       'Year':String,
                       'ZIP':String, #drop
                       'ZIP4':String,#drop
                       'centroid_lat':String,
                       'centroid_long':String}

5. Now convert your expression from the last problem to a function and test that this function works on the first few chunks of each file.

In [14]:
from functoolz import pipeable
read_parcel = lambda path: pd.read_csv(path, chunksize = 500, sep = '|', dtype={'centroid_lat':str, 'centroid_long':str})
read_parcel('./data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt')

In [15]:
addcolumns = pipeable(lambda chunk: (chunk 
                                         >> select(common_columns)
                                         >> mutate(lat_long = pd.Series(zip(chunk.centroid_lat, chunk.centroid_long)))
                                         >> mutate(lake_code = recode(X.lat_long, latlong_to_code))
                                         >> mutate(lake_name = recode(X.lat_long, latlong_to_name))
                                         >> mutate(parcel_dist_to_lake = recode(X.lat_long, latlong_to_distance))
                                         >> filter_by(~X.lake_name.isna())
                                         >> filter_by(X.parcel_dist_to_lake <= 1600)
                            )     )    

In [16]:
ex = read_parcel(files[2])
next(ex).head()

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,...,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_long,centroid_lat
0,0.0,0.16,NaN,NaN,N,Y,14195.0,1,ANDOVER,ANDOVER,...,NaN,NaN,NaN,NaN,2000.0,2006,55304.0,4187.0,-93.26607,45.22905
1,0.0,0.15,NaN,NaN,N,Y,14189.0,1,ANDOVER,ANDOVER,...,NaN,NaN,NaN,NaN,1999.0,2006,55304.0,4187.0,-93.26591,45.22892
2,0.0,0.14,NaN,NaN,N,Y,14177.0,1,ANDOVER,ANDOVER,...,NaN,NaN,NaN,NaN,1999.0,2006,55304.0,4187.0,-93.26566,45.22864
3,0.0,0.16,NaN,NaN,N,Y,14165.0,1,ANDOVER,ANDOVER,...,NaN,NaN,NaN,NaN,1999.0,2006,55304.0,4187.0,-93.26547,45.22829
4,0.0,0.13,NaN,NaN,N,Y,14159.0,1,ANDOVER,ANOKA,...,NaN,NaN,NaN,NaN,2000.0,2006,55304.0,4187.0,-93.26548,45.22811


In [17]:
addcolumns(firstchunk)

,OWN_ADD_L2,YEAR_BUILT,COOLING,GREEN_ACRE,PARC_CODE,EMV_TOTAL,SALE_DATE,TAX_ADD_L2,CITY_USPS,USE4_DESC,...,STREETTYPE,OWNER_NAME,centroid_lat,PREFIX_DIR,STREETNAME,OWN_ADD_L3,lat_long,lake_code,lake_name,parcel_dist_to_lake
42707,OAKDALE,0.0,NaN,N,0.0,209900.0,NaN,OAKDALE,NaN,NaN,...,NaN,NaN,45.21007,NaN,NaN,"MN, 55128","(45.21007, -93.0241)",02000400-01,Peltier Lake,777.247256
42708,HUGO,0.0,NaN,N,0.0,141520.0,2001-06-05,HUGO,NaN,NaN,...,NaN,NaN,45.20913,NaN,NaN,"MN, 55038","(45.20913, -93.03251)",02000400-01,Peltier Lake,777.247256
42709,HUGO,0.0,NaN,N,0.0,269290.0,1997-04-01,HUGO,NaN,NaN,...,NaN,NaN,45.20933,NaN,NaN,"MN, 55038","(45.20933, -93.03805)",02000400-01,Peltier Lake,777.247256
42710,STACY,0.0,NaN,N,0.0,14700.0,1991-06-08,STACY,NaN,NaN,...,NaN,NaN,45.20952,NaN,NaN,"MN, 55079","(45.20952, -93.0438)",02000400-01,Peltier Lake,777.247256
42711,ANOKA,0.0,NaN,N,0.0,148960.0,2003-10-02,ANOKA,NaN,NaN,...,NaN,NaN,45.20919,NaN,NaN,"MN, 55303","(45.20919, -93.04851)",02000400-01,Peltier Lake,777.247256
42712,ANOKA,0.0,NaN,N,0.0,147940.0,2003-10-02,ANOKA,NaN,NaN,...,NaN,NaN,45.20927,NaN,NaN,"MN, 55303","(45.20927, -93.05237)",02000400-01,Peltier Lake,777.247256
42713,ANOKA,0.0,NaN,N,0.0,147940.0,2003-10-02,ANOKA,NaN,NaN,...,NaN,NaN,45.2091,NaN,NaN,"MN, 55303","(45.2091, -93.05506)",02000400-01,Peltier Lake,777.247256
42714,HUGO,0.0,NaN,N,0.0,154700.0,NaN,HUGO,NaN,NaN,...,NaN,NaN,45.21015,NaN,NaN,"MN, 55038","(45.21015, -93.05759)",02000400-01,Peltier Lake,777.247256
42715,ST PAUL,0.0,NaN,N,0.0,7200.0,NaN,ST PAUL,NaN,NaN,...,NaN,NaN,45.20923,NaN,NaN,"MN, 55101","(45.20923, -93.06032)",02000400-01,Peltier Lake,777.247256
42716,ST PAUL,0.0,NaN,N,0.0,771215.0,NaN,ST PAUL,NaN,NaN,...,NaN,NaN,45.20659,NaN,NaN,"MN, 55101","(45.20659, -93.06247)",02000400-01,Peltier Lake,777.247256


In [18]:
ch = first(fileiterator[4])
addcolumns(ch)

AttributeError: 'TextFileReader' object has no attribute 'iloc'

In [ ]:
ch2 = next(fileiterator[5])
addcolumns(ch2)

6. We need to make a unique primary key for each row in the combined parcel file.<br>
    a. There is a column that appears to be a unique parcel id.  Double check that this is a true primary key for each individual file. (To do this you need to verify that the number of unique values is the same as the number of rows for each of the parcel files.  **Hint:** For each file, use of the accumulator pattern with two accumualtors (one number and one data frame). <br>
    b. Explain why this column will not work as a primary key if we want to combine all years in one database. <br>
    c. Suppose we make a new column that consist of `str(year) + '-' + PIN`.  Explain why this should make a proper primary key for the combined data. <br>

In [ ]:
firstchunks = [first(f) for f in fileiterator]

In [107]:
nrows = 0
unique_list = []
for ch in firstchunks:
    nrows = nrows + len(ch)
    print(nrows)
    for x in ch['PIN']:
        if x not in unique_list:
            unique_list.append(x)
    #print(sum(unique_list))    
    #pinset = set(ch['PIN'])
    #unique = unique 
    #unique = unique + ch['PIN'].intersection(unique)
    #unique = pinset.intersection(pinset)
    #print(unique)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500


In [135]:
#[(col, exp[col].is_unique) for col in exp]

This columm will not work as a primary key if we want to combine all the years because there isn't a difference between each year, we can't tell them apart.

str(year) + '-' + PIN should make a proper primary key because adding the year will make the key unique.

7. Make a function to add the key suggested in the last problem (`str(year) + '-' + PIN`) to a given chunk.

In [ ]:
add_primary_key = pipeable(lambda start, df: (df
                                              >> mutate(id = np.arange(start, start + len(df))
                                              )))

In [ ]:
c_size = 50000
from dfply import head
i = 0
(firstchunk 
 >> add_primary_key(i*c_size) 
 >> head(3))

In [ ]:
process_chunk = pipeable(lambda i, df, chunksize=c_size: df >> addcolumns >> add_primary_key(i*c_size))
firstchunk >> process_chunk(0) >> tail(1)

In [ ]:
from more_sqlalchemy import get_sql_types
i = 0
complete_first_chunk = firstchunk >> process_chunk(i)
sql_types = get_sql_types(complete_first_chunk)
#sql_types

In [ ]:
files

In [ ]:
rows_so_far = 0
nrows = rows_so_far
for f in files:
    c_size = 50000
    print('processing file {0}'.format(f))
    df_iter = enumerate(pd.read_csv(f, 
                                chunksize = c_size, sep = '|', 
                                    dtype={'centroid_lat':str, 'centroid_long':str}))
    for i, chunk in df_iter:
        processed_chunk = chunk >> process_chunk(nrows)
        nrows = nrows + len(chunk)

#### Note: If you are clever, you can do parts 8 in one double loop, which will save you from having to read the parcel files twice.

8. It is probably worth our time to test that our new key column is truely unique. (If not, we might be wasting out time loading the data into a database, only to have process fail hours in.) Test that the new column works by <br>
    a. Iterating over all the files.<br>
    b. Using an accumulator to count total number of rows across all parcel files. <br>
    c. Using an accumulator to accumulate a set of all unique values of our new key. <br>
    d. Verifying that we have as many total rows as unique keys.
    a. Selecting just this column. <br>
    b. Dumping this column into a temporary database <br>

9. If the last step succeeded, you can proceed to make a master parcel data database.  If not, you will need to figure out another primary key, probably an `id` column similar to the example in the lectures.